## **LangChain**



- Framework para desarrollar apps basadas en LLMs

### Instalación

In [ ]:
! pip install langchain
! pip install --upgrade --quiet  langchain
! pip install langchain-community
! pip install langchainhub
! pip install langchain-openai
! pip install chromadb bs4

In [82]:
from dotenv import load_dotenv
load_dotenv()
import os
key = os.getenv("OPENAI_API_KEY")

### **Componentes : LLM**

- OpenAI: Chat Model

Inicia el modelo

In [85]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [86]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [87]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Write a recipe using {ingredient}.")

chain = prompt | llm

In [88]:
response = chain.invoke({"ingredient": "tomato"}).content

In [89]:
print(response)

Recipe: Tomato Basil Bruschetta

Ingredients:
- 4 ripe tomatoes, diced
- 1/4 cup fresh basil leaves, chopped
- 2 cloves garlic, minced
- 2 tablespoons extra virgin olive oil
- 1 tablespoon balsamic vinegar
- Salt and pepper to taste
- Baguette or crusty bread, sliced
- Optional: Mozzarella cheese, sliced

Instructions:
1. Preheat the oven to 375°F (190°C). Place the sliced baguette or bread on a baking sheet and toast in the oven until lightly golden and crispy, about 8-10 minutes. Remove from the oven and set aside.

2. In a medium bowl, combine the diced tomatoes, chopped basil, minced garlic, olive oil, and balsamic vinegar. Mix well to combine all the flavors. Season with salt and pepper to taste.

3. If desired, spread a thin layer of sliced mozzarella cheese on each toasted bread slice. Return the bread to the oven for a few minutes until the cheese melts and starts to bubble.

4. Spoon the tomato basil mixture generously over each bread slice. If you skipped the cheese, you can 

- OpenAI: Embeddings Model

In [90]:
from langchain_openai import OpenAIEmbeddings

In [91]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [92]:
# Embed the docs or text (in this case it will be the tomato recipe)
text = response 
doc_result = embeddings.embed_documents([text])
doc_result[0][:5]

[-0.019610895550667983,
 -0.03274273617843824,
 -0.011946397332724516,
 0.040325479172545385,
 -0.01069963855112415]

In [93]:
# Embed the query
query = "How many ingredients will I need for this recipe?"
query_result = embeddings.embed_query(text)
query_result[:5]

[-0.019610895550667983,
 -0.03274273617843824,
 -0.011946397332724516,
 0.040325479172545385,
 -0.01069963855112415]

### **Componentes : Chains**


### **Q&A : Indexing**

- Cargar texto (documentos, base de conocimiento)
- Trocear el texto para poder hacer llamadas a la API
- Almacenar el contenido

In [81]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import PyPDFDirectoryLoader

In [92]:
# Load the documents

loader = PyPDFDirectoryLoader("documents/")
docs = loader.load()

In [ ]:
print(docs[0].page_content)

In [93]:
# Chunk and embed the documents

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

### **VectorStores**

In [ ]:
%pip install --upgrade --quiet  qdrant-client

In [100]:
from langchain_community.vectorstores import Qdrant

In [101]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
vdb = Qdrant.from_documents(
    chunks,
    embeddings,
    url="http://localhost:6333/",
    api_key="12345",
    collection_name="test",
)

In [95]:
# Retrieve and generate using the relevant snippets of the blog.
template = '''You are an assistant for question-answering tasks. Use the following pieces of
        retrieved context to answer the question. If you don't know the answer, just say 
        that you don't know. Use three sentences maximum and keep the answer concise.
        Question: {question} 

        Context: {context} 

        Answer:
        '''

prompt_text = ChatPromptTemplate.from_template(template)


In [96]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [87]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [97]:
retriever = vdb.as_retriever()

In [98]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_text
    | llm
    | StrOutputParser()
)

In [ ]:
# Invoke the chain to get the response
chain.invoke("¿Qué me puedes decir acerca del contexto?")

### **LangChain Components: Chains**

- secuencias de llamadas, ya sea a un LLM, una herramienta o un paso de preprocesamiento de datos
- LCEL facilita la creación de cadenas complejas a partir de componentes básicos

In [102]:
# prompt + model + output parser Chain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("Write a recipe using the ingredient {ingredient}")
model = ChatOpenAI(model_name= "gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

choco_recipe = chain.invoke({"ingredient": "chocolate"})

In [41]:
print (choco_recipe)

Recipe: Chocolate Banana Bread

Ingredients:
- 1 ½ cups all-purpose flour
- 1 teaspoon baking powder
- ½ teaspoon baking soda
- ½ teaspoon salt
- ¼ cup unsweetened cocoa powder
- ½ cup granulated sugar
- ½ cup unsalted butter, softened
- 2 ripe bananas, mashed
- 2 large eggs
- 1 teaspoon vanilla extract
- ½ cup milk
- 1 cup chocolate chips

Instructions:

1. Preheat your oven to 350°F (175°C). Grease and flour a loaf pan.

2. In a medium-sized bowl, whisk together the flour, baking powder, baking soda, salt, and cocoa powder until well combined. Set aside.

3. In a large mixing bowl, cream together the softened butter and granulated sugar until light and fluffy.

4. Add the mashed bananas, eggs, and vanilla extract to the butter mixture. Beat until well incorporated.

5. Gradually add the dry ingredients to the wet mixture, alternating with the milk. Begin and end with the dry ingredients. Mix until just combined, being careful not to overmix.

6. Gently fold in the chocolate chips int

In [42]:
prompt_value = prompt.invoke({"ingredient": "chocolate"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='Write a recipe using the ingredient chocolate')])

In [43]:
prompt_value.messages[0].content

'Write a recipe using the ingredient chocolate'

In [44]:
message = model.invoke(prompt_value)
message

AIMessage(content='Recipe: Decadent Chocolate Brownies\n\nIngredients:\n- 1 cup unsalted butter\n- 2 cups granulated sugar\n- 4 large eggs\n- 1 teaspoon vanilla extract\n- 1 cup all-purpose flour\n- 1/2 cup cocoa powder\n- 1/2 teaspoon baking powder\n- 1/4 teaspoon salt\n- 1 cup chocolate chips (semi-sweet or dark)\n- Optional: 1/2 cup chopped nuts (walnuts or pecans)\n\nInstructions:\n\n1. Preheat your oven to 350°F (175°C). Grease and flour a 9x13-inch baking dish.\n\n2. In a microwave-safe bowl, melt the butter in the microwave or on the stovetop until fully melted. Allow it to cool slightly.\n\n3. In a large mixing bowl, combine the melted butter and granulated sugar. Stir until well combined.\n\n4. Add the eggs one at a time, mixing well after each addition. Stir in the vanilla extract.\n\n5. In a separate bowl, whisk together the flour, cocoa powder, baking powder, and salt.\n\n6. Gradually add the dry ingredients to the wet ingredients, mixing until just combined. Do not overmix

In [45]:
message.content

'Recipe: Decadent Chocolate Brownies\n\nIngredients:\n- 1 cup unsalted butter\n- 2 cups granulated sugar\n- 4 large eggs\n- 1 teaspoon vanilla extract\n- 1 cup all-purpose flour\n- 1/2 cup cocoa powder\n- 1/2 teaspoon baking powder\n- 1/4 teaspoon salt\n- 1 cup chocolate chips (semi-sweet or dark)\n- Optional: 1/2 cup chopped nuts (walnuts or pecans)\n\nInstructions:\n\n1. Preheat your oven to 350°F (175°C). Grease and flour a 9x13-inch baking dish.\n\n2. In a microwave-safe bowl, melt the butter in the microwave or on the stovetop until fully melted. Allow it to cool slightly.\n\n3. In a large mixing bowl, combine the melted butter and granulated sugar. Stir until well combined.\n\n4. Add the eggs one at a time, mixing well after each addition. Stir in the vanilla extract.\n\n5. In a separate bowl, whisk together the flour, cocoa powder, baking powder, and salt.\n\n6. Gradually add the dry ingredients to the wet ingredients, mixing until just combined. Do not overmix.\n\n7. Fold in th

In [46]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nRecipe: Chocolate Chip Cookies\n\nIngredients:\n- 1 cup all-purpose flour\n- 1/2 tsp baking soda\n- 1/2 tsp salt\n- 1/2 cup unsalted butter, softened\n- 1/2 cup granulated sugar\n- 1/4 cup brown sugar\n- 1 egg\n- 1 tsp vanilla extract\n- 1 cup chocolate chips\n\nInstructions:\n\n1. Preheat your oven to 375°F (190°C) and line a baking sheet with parchment paper.\n2. In a medium bowl, whisk together the flour, baking soda, and salt. Set aside.\n3. In a separate large bowl, cream together the softened butter, granulated sugar, and brown sugar until light and fluffy.\n4. Add in the egg and vanilla extract, and mix until well combined.\n5. Gradually add in the dry ingredients to the wet mixture, mixing until a dough forms.\n6. Fold in the chocolate chips until evenly distributed throughout the dough.\n7. Using a spoon or cookie scoop, drop the dough onto the prepared baking sheet, leaving about 2 inches of space between each cookie.\n8. Bake for 10-12 minutes, or until the edges are go

### **Tools**

- interfaces que un Agent puede usar para interactuar con el mundo.
    - Nombre
    - Descripción
    - Esquema JSON de cuáles son los datos de entrada
    - Función a llamar
    - Si el resultado debe ser devuelto directamente al usuario







In [ ]:
%pip install --upgrade --quiet  youtube_search

In [127]:
# Built in tools
from langchain.tools import YouTubeSearchTool

tool = YouTubeSearchTool()

In [133]:
tool.name

'youtube_search'

In [134]:
tool.description

'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [112]:
tool.run("john green, 5")

"['https://www.youtube.com/watch?v=Dc0gPrpdlTc&pp=ygUKam9obiBncmVlbg%3D%3D', 'https://www.youtube.com/watch?v=C2fsMsPz1DQ&pp=ygUKam9obiBncmVlbg%3D%3D', 'https://www.youtube.com/watch?v=MSYw502dJNY&pp=ygUKam9obiBncmVlbg%3D%3D', 'https://www.youtube.com/watch?v=jNEUz9v5RYo&pp=ygUKam9obiBncmVlbg%3D%3D', 'https://www.youtube.com/watch?v=YbyeApgIiBk&pp=ygUKam9obiBncmVlbg%3D%3D']"

In [137]:
# Define Custom tools
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

multiply.name
multiply.description
multiply.invoke({"a": 2, "b": 3})

6

### **Agent**

- es la cadena responsable de decidir qué paso dar a continuación. Por lo general, esto funciona con un modelo de lenguaje, un mensaje y un output parser.
- el agent executor es el runtime de un agente. Esto es lo que realmente llama al agente, ejecuta las acciones que elige, devuelve los resultados de la acción al agente y repite.

In [53]:
# Build custom Agent

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [116]:
# Define a helper function for custom tool
def is_palindrome(word):
    return word == word[::-1]

In [152]:
from langchain.agents import tool

# Define a custom tool: is_palindrome_tool
@tool
def is_palindrome_tool(word: str) -> bool:
    """Check if a word is a palindrome."""
    return is_palindrome(word) 


is_palindrome_tool.invoke("abc")

False

In [154]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

get_word_length.invoke("hello")

5

In [119]:
# Define a list of tools for the agent to use
tools = [is_palindrome_tool, get_word_length]

In [138]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [142]:
# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

In [143]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [144]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [157]:
list(agent_executor.stream({"input": "{¿La palabra chocolate es palíndromo?}"}))	



> Entering new AgentExecutor chain...

Invoking: `is_palindrome_tool` with `{'word': 'chocolate'}`


FalseNo, la palabra "chocolate" no es un palíndromo.

> Finished chain.


[{'actions': [OpenAIToolAgentAction(tool='is_palindrome_tool', tool_input={'word': 'chocolate'}, log="\nInvoking: `is_palindrome_tool` with `{'word': 'chocolate'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_JOczmwzHgmuyP9KN267fNUsY', 'function': {'arguments': '{\n  "word": "chocolate"\n}', 'name': 'is_palindrome_tool'}, 'type': 'function'}]})], tool_call_id='call_JOczmwzHgmuyP9KN267fNUsY')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_JOczmwzHgmuyP9KN267fNUsY', 'function': {'arguments': '{\n  "word": "chocolate"\n}', 'name': 'is_palindrome_tool'}, 'type': 'function'}]})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='is_palindrome_tool', tool_input={'word': 'chocolate'}, log="\nInvoking: `is_palindrome_tool` with `{'word': 'chocolate'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_JOczmwzHgmuyP

### **Q&A Agent**

In [158]:
# Vector Store
vdb
# Retriever
retriever = vdb.as_retriever()

In [159]:
# Define Retriever Tool

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "search_LinkedIn_profiles",
    "Searches and returns pieces of text from LinkedIn profiles.\n",
)

tools = [retriever_tool]

In [160]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [181]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

retriever_agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=retriever_agent, tools=tools)

In [182]:
response = agent_executor.invoke({"input": "Hola, me llamo Diana"})



> Entering new AgentExecutor chain...
Hola Diana, ¿en qué puedo ayudarte hoy?

> Finished chain.


In [183]:
response = agent_executor.invoke({"input": "¿Sabes quién es Diana Diaz Bisbe?"})



> Entering new AgentExecutor chain...



Invoking: `search_LinkedIn_profiles` with `{'query': 'Diana Diaz Bisbe'}`


Contact
diana.bisbe@gmail.com
www.linkedin.com/in/
dianadiazbisbe  (LinkedIn)
Top Skills
Semantic Kernel
.NET Framework
Back-End Development
Certifications
Power BI Data Modeling with DAX
Global AI Bootcamp Participant
Intro to GitHub Copilot Course
Intro to Generative AI Course
BI Dashboards with Power BI
CourseDiana Diaz Bisbe
Cloud Solutions Developer @ ENCAMINA | Computer Science
Madrid, Community of Madrid, Spain
Summary
Diana Diaz Bisbe is a software engineer dedicated to the world of
artificial intelligence. 
Her passion for technology and her ability to solve complex technical
problems are evident in her work, where she primarily uses Python
and C# to develop impactful applications. Diana holds a degree in
Computer Science from the University of Central Florida. She is an
avid advocate for lifelong learning and stands out as a speaker in the
artificial intelligence community. She has held roles as a de

In [184]:
response["output"]

'Diana Diaz Bisbe es una desarrolladora de soluciones en la nube en ENCAMINA, con experiencia en inteligencia artificial. Su pasión por la tecnología y su habilidad para resolver problemas técnicos complejos se reflejan en su trabajo, donde utiliza principalmente Python y C# para desarrollar aplicaciones impactantes. Diana tiene un título en Ciencias de la Computación de la Universidad de Central Florida. También es una defensora del aprendizaje continuo y se destaca como ponente en la comunidad de inteligencia artificial.'